In [74]:
import requests
import json
from urllib.parse import urlencode
import re
import time

In [73]:
base_url = 'https://m.weibo.cn/api/container/getIndex?'
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'Host':'m.weibo.cn',
    'Referer':'https://m.weibo.cn/u/1752054291',
    'X-Requested-With': 'XMLHttpRequest'
}

params = {
    'type': 'uid',
    'value': '1752054291',
    'containerid': '1076031752054291',
    'page':1
}
url = base_url + urlencode(params)
r = requests.get(url,headers)
r

<Response [200]>

定义获取JSON数据的函数

In [36]:
def get_data(index):
    params = {
    'type': 'uid',
    'value': '1752054291',
    'containerid': '1076031752054291',
    'page':index
    }
    url = base_url + urlencode(params)
    try:
        r = requests.get(url,headers)
        if r.status_code == 200:
            return r.json()
    except requests.ConnectionError as e:
        print("error",e.args)

定义处理JSON数据并且返回迭代器的函数

In [61]:
def parse_data(j_data):
    if j_data:
        its = j_data.get('data').get('cards')
        for it in its:
            if it.get('card_type') != 9:
                continue
            it = it.get('mblog')
            weibo = {}
            weibo['id'] = it.get('id')
            weibo['text'] = pq(it.get('text')).text()
            weibo['reposts'] = it.get('reposts_count')
            weibo['comments'] = it.get('comments_count')
            weibo['attitudes'] = it.get('attitudes_count')
            weibo['pic'] = it.get('original_pic')
            yield weibo

定义写文件的函数

In [77]:
def write_data(weibos):
    with open("weibo.md",'a+',encoding='utf-8') as f:
        f.write('# XXX\n')
        for x in weibos:
            f.write(x['text']+'\n')
            f.write('点赞数:'+str(x['attitudes'])+' 转发数:'+str(x['reposts'])+ ' 评论数:'+str(x['comments'])+'\n')
            f.write('![](%s)\n\n' % x['pic'])
        print('write ok!')

In [79]:
for index in range(1,5):
    j_data = get_data(index)
    result = parse_data(j_data)
    write_data(result)
    time.sleep(2)

write ok!
write ok!
write ok!
write ok!


In [41]:
j_data = r.json()

In [55]:
its = j_data.get('data').get('cards')
weibos = []
for it in its:
    if it.get('card_type') != 9:
        continue
    it = it.get('mblog')
    weibo = {}
    weibo['id'] = it.get('id')
    weibo['text'] = pq(it.get('text')).text()
    weibo['reposts'] = it.get('reposts_count')
    weibo['comments'] = it.get('comments_count')
    weibo['attitudes'] = it.get('attitudes_count')
    weibo['pic'] = it.get('original_pic')
    weibos.append(weibo)

In [56]:
for x in weibos:
    print(x)

{'id': '4462943929991658', 'text': '#晚安#', 'reposts': 2, 'comments': 0, 'attitudes': 31, 'pic': 'http://wx1.sinaimg.cn/large/686e3a13ly1gb3f51nc0ij20jg0axq3t.jpg'}
{'id': '4462920761104649', 'text': '这世界上只有一种成功：\n用自己喜欢的方式，\n度过自己的一生。', 'reposts': 8, 'comments': 3, 'attitudes': 76, 'pic': 'http://wx2.sinaimg.cn/large/686e3a13ly1gb3cgkv486j20jg09lmxy.jpg'}
{'id': '4462873306581513', 'text': '#灵魂拷问#\n\n人生究竟是为了什么呀', 'reposts': 8, 'comments': 10, 'attitudes': 38, 'pic': 'http://wx4.sinaimg.cn/large/686e3a13ly1gb370p682qj20i10fbdhf.jpg'}
{'id': '4462846223613683', 'text': '想不负此生真的很难，\n只希望你们能过上自己想要的日子，\n健健康康的。\n\n——#电影后来的我们#', 'reposts': 3, 'comments': 1, 'attitudes': 56, 'pic': 'http://wx3.sinaimg.cn/large/686e3a13ly1gb33v4ti52j20jg0jc7cr.jpg'}
{'id': '4462722093576074', 'text': '#早啊#\n\n我今年和明年的愿望都是：\n变美变瘦\n变有钱', 'reposts': 12, 'comments': 2, 'attitudes': 97, 'pic': 'http://wx3.sinaimg.cn/large/686e3a13ly1gb2pn72eu7j20jg0zcznx.jpg'}
{'id': '4462562583783721', 'text': '#晚安#\n\n“所谓幸福”：见图', 'rep

In [60]:
with open("电影台词weibo.md",'w',encoding='utf-8') as f:
    for x in weibos:
        f.write(x['text']+'\n')
        f.write('点赞数:'+str(x['attitudes'])+' 转发数:'+str(x['reposts'])+ ' 评论数:'+str(x['comments'])+'\n')
        f.write('![](%s)\n\n' % x['pic'])
    print('write ok!')

write ok!


In [47]:
ind = its[0]
ind['card_type'] == 11

True

In [49]:
ind.get('card_type') == 11

True

转化为JSON数据

In [35]:
data = json.loads(r.text)
data

{'ok': 1,
 'data': {'cardlistInfo': {'containerid': '1076031752054291',
   'v_p': 42,
   'show_style': 1,
   'total': 1277,
   'since_id': 4461790798805381},
  'cards': [{'card_type': 9,
    'itemid': '1076031752054291_-_4462377338811783',
    'scheme': 'https://m.weibo.cn/status/IqaIBAYlx?mblogid=IqaIBAYlx&luicode=10000011&lfid=1076031752054291',
    'mblog': {'visible': {'type': 0, 'list_id': 0},
     'created_at': '昨天 09:25',
     'id': '4462377338811783',
     'idstr': '4462377338811783',
     'mid': '4462377338811783',
     'can_edit': False,
     'show_additional_indication': 0,
     'photoTag': [{'picid': '686e3a13ly1gb1m1csw7tj20jg0auq3x',
       'hastag': True,
       'taginfo': {'code': '100000',
        'msg': '',
        'data': {'4462377338811783': {'pic_object_id': '1042018:5b8fe474a7b9d484f935671db75eaa41',
          'photo_id': 4462377338811783,
          'mid': 4462377338811783,
          'pid': '686e3a13ly1gb1m1csw7tj20jg0auq3x',
          'uid': 1752054291,
         

In [30]:
text = json.dumps(data)
text

'{"ok": 1, "data": {"cardlistInfo": {"containerid": "1076031752054291", "v_p": 42, "show_style": 1, "total": 1277, "since_id": 4462377338811783}, "cards": [{"card_type": 11, "card_style": 1, "display_arrow": 0, "skip_group_title": true, "card_group": [{"card_type": 17, "col": 2, "group": [{"title_sub": "#\\u665a\\u5b89#", "word_scheme": "#\\u665a\\u5b89#", "scheme": "https://m.weibo.cn/search?isnewpage=1&disable_history=1&disable_sug=1&extparam=c_type%3D99%26profile_uid%3D1752054291&luicode=10000011&lfid=1076031752054291&containerid=100103type%3D401%26t%3D99%26q%3D%23%E6%99%9A%E5%AE%89%23", "action_log": {"act_code": 3089, "oid": "1752054291", "ext": "scene:profileMblog", "fid": ""}, "icon": "https://simg.s.weibo.com/20190429_new_small.png"}, {"title_sub": "#\\u65e9\\u554a#", "word_scheme": "#\\u65e9\\u554a#", "scheme": "https://m.weibo.cn/search?isnewpage=1&disable_history=1&disable_sug=1&extparam=c_type%3D99%26profile_uid%3D1752054291&luicode=10000011&lfid=1076031752054291&containeri

In [6]:
r.json()

{'ok': 1,
 'data': {'cardlistInfo': {'containerid': '1076031752054291',
   'v_p': 42,
   'show_style': 1,
   'total': 1276,
   'since_id': 4462203514358416},
  'cards': [{'card_type': 11,
    'card_style': 1,
    'display_arrow': 0,
    'skip_group_title': True,
    'card_group': [{'card_type': 17,
      'col': 2,
      'group': [{'title_sub': '#早啊#',
        'word_scheme': '#早啊#',
        'scheme': 'https://m.weibo.cn/search?isnewpage=1&disable_history=1&disable_sug=1&extparam=c_type%3D99%26profile_uid%3D1752054291&luicode=10000011&lfid=1076031752054291&containerid=100103type%3D401%26t%3D99%26q%3D%23%E6%97%A9%E5%95%8A%23',
        'action_log': {'act_code': 3089,
         'oid': '1752054291',
         'ext': 'scene:profileMblog',
         'fid': ''},
        'icon': 'https://simg.s.weibo.com/20190429_new_small.png'},
       {'title_sub': '#晚安#',
        'word_scheme': '#晚安#',
        'scheme': 'https://m.weibo.cn/search?isnewpage=1&disable_history=1&disable_sug=1&extparam=c_type%3D99%

查看json数据的类型，可以是list或者dict

In [22]:
j_data = r.json()
type(j_data)
with open('data.json','w',encoding='utf-8') as f:
    f.write(json.dumps(j_data,indent=2,ensure_ascii=False))
    print("ok!")

ok!


In [16]:
st = 'abc</br>'
l = st.replace('</br>','')
l

'abc'

In [27]:
a = j_data.get('data')
a

{'cardlistInfo': {'containerid': '1076031752054291',
  'v_p': 42,
  'show_style': 1,
  'total': 1277,
  'since_id': 4462377338811783},
 'cards': [{'card_type': 11,
   'card_style': 1,
   'display_arrow': 0,
   'skip_group_title': True,
   'card_group': [{'card_type': 17,
     'col': 2,
     'group': [{'title_sub': '#晚安#',
       'word_scheme': '#晚安#',
       'scheme': 'https://m.weibo.cn/search?isnewpage=1&disable_history=1&disable_sug=1&extparam=c_type%3D99%26profile_uid%3D1752054291&luicode=10000011&lfid=1076031752054291&containerid=100103type%3D401%26t%3D99%26q%3D%23%E6%99%9A%E5%AE%89%23',
       'action_log': {'act_code': 3089,
        'oid': '1752054291',
        'ext': 'scene:profileMblog',
        'fid': ''},
       'icon': 'https://simg.s.weibo.com/20190429_new_small.png'},
      {'title_sub': '#早啊#',
       'word_scheme': '#早啊#',
       'scheme': 'https://m.weibo.cn/search?isnewpage=1&disable_history=1&disable_sug=1&extparam=c_type%3D99%26profile_uid%3D1752054291&luicode=10000

In [23]:
!pip install pyquery

In [25]:
from pyquery import PyQuery as pq

In [26]:
doc = pq(r.text)

In [ ]:
doc('')